In [1]:
A,B,G=0,0,0
isColab=False
if isColab:
    %pip install ultralytics
    import ultralytics
    ultralytics.checks()
    from google.colab.patches import cv2_imshow
    from google.colab import drive
from sqlalchemy.dialects.oracle.dictionary import all_objects
import csv
import pickle
from models.TrackedObject import TrackedObject
from models.VideoAnalysis import VideoAnalysis
from utils.dynamics import EuclideanDistTracker, get_overlap_info
import utils.files as fil
import cv2
import os
import analysis.mask_rcnn as mask_rcnn
from analysis import yolo
from utils.visualise import visualize
%load_ext autoreload
%autoreload 2

from IPython.display import display, clear_output
import PIL.Image

C:\Users\tomfi\master_thesis\models\TrackedObject.py:143: SyntaxWarning: invalid decimal literal
  if beta>=(3/4)*3.15and beta<(4/4)*3.15:


In [28]:

transform, COCO_INSTANCE_CATEGORY_NAMES, model, device= mask_rcnn.config()
model= yolo.config()

def run_detection(frame_filenames, detection_file_path):
    if os.path.exists(detection_file_path):
        print("Detection file already exists. Skipping YOLO detection.")
        return

    print("Running YOLO detection and saving results...")
    all_detections = []
    i=0
    for frame_filename in frame_filenames:
        i+=1
        if i > 14000:
            print(i)
            frame = cv2.imread(frame_filename)
            if frame is None:
                all_detections.append([])  # Append empty detection for consistency
                continue

            detections = yolo.process_frame(model, frame, device, transform,
                                            COCO_INSTANCE_CATEGORY_NAMES,
                                            tracker=None,
                                            conf=0.5)
            all_detections.append(detections)
            if i % 1000 == 0:

                with open(detection_file_path+str(i), 'wb') as f:
                    pickle.dump(all_detections, f)
                    print(f"Saved detections to {detection_file_path+str(i)}")
                    all_detections = []
    with open(detection_file_path+str(i), 'wb') as f:
        pickle.dump(all_detections, f)
        print(f"Saved detections to {detection_file_path+str(i)}")
        all_detections = []



def process_clip(frame_filenames, case_folder, detection_file_path):
    accident_cnt=0
    detected_accidents = set()
    acc_detected=False
    tracker = EuclideanDistTracker()
    va=VideoAnalysis()

    # Load YOLO detections
    if not os.path.exists(detection_file_path):
        raise FileNotFoundError(f"No detection file found at {detection_file_path}")
    with open(detection_file_path, 'rb') as f:
        all_detections = pickle.load(f)


    # Get frame size dynamically from the first frame
    sample_frame = cv2.imread(frame_filenames[0])
    height, width = sample_frame.shape[:2]
    frame_size = (width, height)
    fps = 18

    # Save output to MP4 instead of AVI
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'avc1' for better compatibility
    out = cv2.VideoWriter(case_folder+"_output_maskrcnn.mp4", fourcc, fps, frame_size)

    for idx, frame_filename in enumerate(frame_filenames):
        frame = cv2.imread(frame_filename)
        if frame is None:
            continue
        #result_frame,_,_objects = mask_rcnn.process_frame(model, frame, device, transform, COCO_INSTANCE_CATEGORY_NAMES, tracker, conf=0.5)
        _objects = all_detections[idx]
        boxes = tracker.update(_objects)
        get_overlap_info(boxes)

        for o in va.allObjects:
          o.is_present=False
          o.is_accident=False
        for obj in boxes:
          if obj.id>(len(va.allObjects)-1):
            obj.H=height
            va.allObjects.append(obj)
          else:
            o=va.allObjects[obj.id]
            o.id=obj.id
            o.is_present=obj.is_present
            o.x1=obj.x1
            o.x2=obj.x2
            o.y1=obj.y1
            o.y2=obj.y2
            o.centroid=obj.centroid
            o.overlaps=obj.overlaps
            o.past_centroids.append(obj.centroid)
            o.compute_speed()
            o.compute_acceleration()
            o.compute_vector()
            #print(str(o.id)+str(o.is_accident)+str(o.centroid))
        #va.allObjects=get_overlap_info(va.allObjects)
        for box in va.allObjects:
            box.check_accident(A, B, G)
            if box.is_accident and box.id not in detected_accidents:
                accident_cnt += 1
                detected_accidents.add(box.id)
        result_frame=visualize(frame, va.allObjects)
        #print(frame_filename)
        out.write(result_frame)
        if isColab:
            #Convert to PIL format for proper display in Jupyter/Colab
            #Display only the current frame
            #clear_output(wait=True)
            result_frame_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            result_pil = PIL.Image.fromarray(result_frame_rgb)
            display(result_pil)
        else:
            cv2.imshow('res',result_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                  break
    out.release()
    cv2.destroyAllWindows()
    return acc_detected, accident_cnt


Ultralytics 8.3.40  Python-3.12.7 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
Setup complete  (12 CPUs, 15.7 GB RAM, 440.7/454.3 GB disk)


In [32]:
if isColab:
    drive.mount('/content/drive/',force_remount=True)
    frames = '/content/drive/MyDrive/data/normal/'
else:
    frames = './data/test/'

results=[]
A,B,G,T=-7,0,4,0
for T in [25]:
    folders = os.listdir(frames)
    accident_count, total_count=0,0
    for case_folder in folders:
        frame_filenames= sorted(
            [f for f in os.listdir(frames+case_folder) if f.endswith('.jpg')],
            key=fil.extract_number
        )

        frame_filenames = [os.path.join(frames+case_folder, f) for f in frame_filenames]
        detection_file_path = case_folder + "_detections.pkl"
        if not os.path.exists(detection_file_path):
            run_detection(frame_filenames, detection_file_path)
        else:
            print(f"Detections already exist at {detection_file_path}. Skipping detection.")
        acc_detected, cnt = process_clip(frame_filenames, case_folder, detection_file_path)
        if acc_detected == True:
            accident_count+=1
        total_count+=1
    print ("Total count: ",total_count)
    print ("Accident count: ",accident_count)
    print ("A",A,"B",B,"G",G)

    results.append({
        "A": A,
        "total_count": cnt,
        "accident_count": accident_count,
        "B": B,
        "G": G
    })
csv_output_file = "test_combined30.csv"
with open(csv_output_file, 'w', newline='') as csvfile:
    fieldnames = ["A", "total_count", "accident_count", "B", "G"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

Detections already exist at combined_detections.pkl. Skipping detection.
Loading detections from combined_detections.pkl1000
Loading detections from combined_detections.pkl2000
Loading detections from combined_detections.pkl3000
Loading detections from combined_detections.pkl4000
Loading detections from combined_detections.pkl5000
Loading detections from combined_detections.pkl6000
Loading detections from combined_detections.pkl7000
Loading detections from combined_detections.pkl8000
Loading detections from combined_detections.pkl9000
Loading detections from combined_detections.pkl10000
Loading detections from combined_detections.pkl11000
Loading detections from combined_detections.pkl12000
Loading detections from combined_detections.pkl13000
Loading detections from combined_detections.pkl14000
Loading detections from combined_detections.pkl15000
Loading detections from combined_detections.pkl16000
Loading detections from combined_detections.pkl17000
Loading detections from combined_de

KeyboardInterrupt: 